In [1]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index,d_hkl,three_dot,cross_plane, get_right_hand
from csl_generator import print_list,get_theta_m_n_list, degrees
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file, get_ang_list
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot
from abc import ABCMeta, abstractclassmethod
import copy
import itertools
import collections

In [2]:
class Element2:
    def __init__(self,name,dirname,crystal_structure):
        self.name = name
        self.dirname = dirname
        self.grandenergy = self.getLatE(dirname)
        self.LP = self.getLatP(dirname)
        self.mass = self.get_mass(dirname)
        self.ciffile = glob.glob(f"{dirname}/*.cif")[0]
        self.crystal_structure = crystal_structure # this is a class representing crystal structure
#         with open(f"{self.dirname}/proto.in","r") as f:
#             self.proto_text = f.read() 
    
    def potential(self):
        for dirs in glob.glob(f"{self.dirname}/potential/*"):
            shutil.copy(f"{dirs}",f"{os.getcwd()}")
        
#     def proto(self):
#         shutil.copy(f"{self.dirname}/proto.in",f"{os.getcwd()}")
    
    @staticmethod
    def getLatE(dirname):
        """
        read the Energy of single crystal 
        """
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatE = lines[-4:][0].split()[4].replace(';', '')
        return float(LatE)

    @staticmethod
    def getLatP(dirname):
        """
        read the Lattice Parameter
        """
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatP = lines[-5:][0].split()[4].replace(';', '')
        return float(LatP)

    @staticmethod
    def get_mass(dirname):
        with open(f'{dirname}/mass','r') as f:
            lines=f.read()
        return float(lines)

In [3]:
class ProtoinCreator(metaclass=ABCMeta):
    # element →　Element クラス　オブジェクト　インスタンス化"前"
    # SanplingMethod　→　SamplingMethodCreator クラス　オブジェクト　インスタンス化"前"
    def __init__(self,element,SamplingMethod):
        self.element = element
        self.sampling_method = SamplingMethod
        self.vanishing_rule = element.crystal_structure.vanishing_rule # this is function
#         self.crystal_structurte = self.element.crystal_structure
        # で、僕はこのself.crystal_structureが持っているメソッドを使いたい
    def generate(self,axis,hkl):
        samplingmethod = self.sampling_method(axis,hkl) # インスタンス化後のSamplingMethodCreatorクラスメソッド
        samplingmethod.vanishing_rule = self.vanishing_rule # passしていたvanishing_ruleをオーバーライド
        protoinbodycreator = ProtoinBodyCreator(samplingmethod)
        self.body_text = protoinbodycreator.generate()
    # サブクラスがインスタンス化されるまで保留して一旦設計を進めることができる
    
    @abstractclassmethod
    def header(self):
        pass
    
    @abstractclassmethod
    def body(self):
        pass 
    
    
    #テンプレートの役割を持つクラス
    def create(self):
        h = self.header()
        b = self.body()
        return f"{h}\n\n{b}"

In [4]:
class Protoin(ProtoinCreator):
    
    # 継承先を固定しておけばテンプレートを動かさなくていい
    def header(self):
        text = f"""


variable element_mass equal {self.element.mass} 

variable minimumenergy equal {self.element.grandenergy}

variable lattice_param equal {self.element.LP}
                """
        
        return text

    
    def body(self):
        return self.body_text
    

In [5]:
class ProtoinBodyCreator:
    def __init__(self,sampling_method_creator):
        self.sampling = sampling_method_creator
        
    def generate(self):
        return self.sampling.calc_condition()

In [6]:
class Calc_Condition:
    def __init__(self,element,SamplingMethod,cell_size,fix_bulk_region,middle_region,start_bulk,end_bulk):

#         self.cnid_sampling = cnid_sampling
        self.element = element
        self.protoin = Protoin(self.element,SamplingMethod)
        self.cell_size = cell_size
        self.fix_bulk_region = fix_bulk_region
        self.middle_region = middle_region
        self.start_bulk = start_bulk
        self.end_bulk = end_bulk
        
    def define_bicrystal_regions(self,xhi):
        tol = 1e-5
        """
        generate a file defining some regions in the LAMMPS and define the atoms
        inside these regions into some groups.
        argument:
        region_names --- list of name of regions
        region_los --- list of the low bounds
        region_his --- list of the hi bounds
        """
        end_fixbulk1 = xhi/2-self.fix_bulk_region
        start_fixbulk2 = xhi/2+self.fix_bulk_region
        start_middle = xhi/2-self.middle_region
        end_middle = xhi/2+self.middle_region
        start_right = xhi/2 - tol
        start_bulk = self.start_bulk
        end_bulk = self.end_bulk


        with open('blockfile', 'w') as fb:
            fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(end_fixbulk1))
            fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_fixbulk2))
            fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_middle,end_middle))
            fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_bulk,end_bulk))
            fb.write('group fixbulk1 region fixbulk1 \n')
            fb.write('group fixbulk2 region fixbulk2 \n')
            fb.write('group middle region middle \n')
            fb.write('group right region right \n')
            fb.write("group left region left \n")
            fb.write('group bulk region bulk \n')
        
    def define_bicrystal_regions2(self,xhi):
        half_cell = xhi/2
        text = f"""

    displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

    group left_dash region left 

    group deleted_atoms subtract left_dash left

    displace_atoms deleted_atoms move {half_cell} 0 0 units box

    group fixbulk1_dash region fixbulk1 
    group fixbulk2_dash region fixbulk2 
    group middle_dash region middle 
    group bulk_dash region bulk
                """
        with open("blockfile2", "w") as f:
            f.writelines(text)
        
    def generate_files(self,axis,hkl,xhi):
        self.protoin.generate(axis,hkl)
        self.element.potential()
        self.define_bicrystal_regions(xhi)
        self.define_bicrystal_regions2(xhi)
        with open("proto.in","w") as f:
            f.writelines(self.protoin.create())
        
#         if self.cnid_sampling==True:
#             self.element.proto()
#         else:
#             with open("proto.in","w") as f:
#                 f.writelines(self.protoin.create())

In [7]:
class core2(core):
    def __init__(self, file_1, file_2):
        super().__init__(file_1,file_2)
    def compute_stgb(self, axis, plane_n, lim = 20, tol = 1e-10):
        """
        compute the transformation to obtain the supercell of the two slabs forming a interface
        argument:
        hkl --- miller indices of the plane expressed in lattice_1
        lim --- the limit searching for a CSL vector cross the plane
        normal_ortho --- whether limit the vector crossing the GB to be normal to the GB
        plane_ortho --- whether limit the two vectors in the GB plane to be orthogonal
        tol --- tolerance judging whether orthogonal
        """
        hkl = MID(self.lattice_1,plane_n)
        self.d1 = d_hkl(self.lattice_1, hkl)
        lattice_2 = three_dot(self.R, self.D, self.lattice_2)
        hkl_2 = get_primitive_hkl(hkl, self.lattice_1, lattice_2)
        self.d2 = d_hkl(lattice_2, hkl_2)
        hkl_c = get_primitive_hkl(hkl, self.lattice_1, self.CSL) # miller indices of the plane in CSL
        hkl_c = np.array(hkl_c)
        
        #plane_n = dot(self.lattice_1,hkl)
        axis_cartesian = dot(self.conv_lattice_1,axis)
        v1 = cross_plane(lattice=self.CSL, n=plane_n, lim=lim,  orthogonal=True, tol=tol) # a CSL basic vector cross the plane
        v2 = cross_plane(lattice=self.CSL, n=axis_cartesian, lim=lim, orthogonal=True, tol=tol)
        v3_cartesian = cross(v1,v2)
        v3 = cross_plane(lattice=self.CSL, n=v3_cartesian, lim=lim, orthogonal=True, tol=tol)
        supercell = np.column_stack((v1,v2,v3)) # supercell of the bicrystal
        supercell = get_right_hand(supercell) # check right-handed


        self.bicrystal_U1 = np.array(np.round(dot(inv(self.lattice_1), supercell),8),dtype = int)
        self.bicrystal_U2 = np.array(np.round(dot(inv(self.lattice_2_TD), supercell),8),dtype = int)
        self.cell_calc.compute_CNID(hkl)
        CNID = self.cell_calc.CNID
        self.CNID = dot(self.lattice_1, CNID)

In [8]:
  class MyInterface:
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        
        self.my_interface = core2(self.cif_filename, self.cif_filename)
        
        self.my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=self.max_sigma, sgm2=self.max_sigma, dd = 1e-4)
        factor = self.LP/(norm(self.my_interface.conv_lattice_1[:,0]))
        self.my_interface.lattice_1 =  self.my_interface.lattice_1*factor
        self.my_interface.lattice_2 =  self.my_interface.lattice_2*factor
        self.my_interface.conv_lattice_1 =  self.my_interface.conv_lattice_1*factor
        self.my_interface.conv_lattice_2 =  self.my_interface.conv_lattice_2*factor
        

In [9]:
class Factory(object): # ここはインスタンスを生成する工場だから普段インスタンスを作ることをやる
    def create(self,hkl,delivery_point="."): # インスタンスをcreate
        # 回転軸、回転角、plane_nを受け取って作成
        axis, theta, plane_n = self.get_gb_info_from_hkl(hkl)
        self.createGB(axis,theta,plane_n,hkl,delivery_point)
    
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
        # interfaceインスタンス化 + get_gbfiles実行
        pass

    
    def generate_hkl_list(self,axis,max_sigma):
        pass
    
    def get_gb_info_from_hkl(self,hkl):
        pass
    


In [10]:
class GBFactory(Factory):
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        self.primitive_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        self.conventional_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
    # interface serch csl create_gbfiles実行
        axis_cart = dot(self.primitive_interface.conv_lattice_1, np.array(axis))

        axis_pri = np.array(np.round(dot(inv(self.primitive_interface.lattice_1), axis_cart),8),dtype = int)
        print(axis_pri)
        print(theta)
        print(self.primitive_interface.lattice_1)
        
        self.primitive_interface.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        self.get_gb_files(axis,plane_n,hkl,delivery_point)
        
#         get_gb_files(self,interface,axis,hkl,calc_condition,input_dirname=None)
    
    @classmethod
    def generate_hkl_list(cls,axis,max_sigma,calc_condition):
        conventional_interface = copy.deepcopy(cls(max_sigma,axis,calc_condition).conventional_interface)
        conventional_interface.lattice_1 = conventional_interface.conv_lattice_1
        conventional_interface.lattice_2 = conventional_interface.conv_lattice_2
        
        thetas = cls.get_thetas(axis,max_sigma)

        gb_dict = {}

        for theta in thetas:

            conventional_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
            n_1, n_2 = cls.get_a_b(conventional_interface.CSL, dot(conventional_interface.lattice_1,np.array(axis)))
            
            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_1),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info1 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_1]}
            print(hkl_name)

            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_2),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info2 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_2]}

            if axis==[1,1,1,]:
                gb_dict = {**gb_dict,**gb_info1}
            else:
                gb_dict = {**gb_dict,**gb_info1,**gb_info2}
            
            print(list(gb_dict.keys()))
        
        cls.gb_dict = gb_dict
        
        return cls(max_sigma,axis,calc_condition)
    
    def get_gb_info_from_hkl(self,hkl):
        return self.gb_dict[hkl]
     
    @staticmethod
    def get_thetas(axis,max_sigma):
        theta_list = []
        sigma_list = []
        lists, thetas = GBFactory.getsigmas(axis, max_sigma)
        return thetas
    
    @staticmethod
    def get_csl_basis(CSL,axis):
        tol = 1e-8
        dot_list = get_ang_list(CSL.T,axis)
        normal_v = CSL.T[np.where(abs(dot_list - 1) > tol)[0]]
        return normal_v
    
    @staticmethod
    def get_a_b(CSL, axis):
        hkl_perp_axis = MID(CSL, axis)
        a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T
        
        if (norm(cross(axis,[1,1,1])) < 1e-8):
            v1, v2 = GBFactory.get_csl_basis(CSL,axis)
            if dot(v1,v2) < 0:
                v2 = -v2
        
        if(norm(cross(axis,[1,0,0])) < 1e-8):
            b = a + b
        elif (norm(cross(axis,[1,1,1])) < 1e-8):
            a = v1 + v2
#         elif (norm(cross(axis,[1,1,1])) < 1e-8):
#             if dot(a,b) < 0:
#                 b = a + b
#             b = a + b
#         if (abs(norm(a) - norm(b)) < 1e-8):
#             raise RuntimeError ('the tow vectors are identical!')

        return a.T, b.T
    
    @staticmethod
    def getsigmas(uvw, limit):
        """
        ---by Haddian---
        prints a list of smallest sigmas/angles for a given axis(uvw).
        """
        sigmas = []
        thetas = []
        for i in range(limit):

            tt = get_theta_m_n_list(uvw, i)
            if len(tt) > 0 and i > 1:
                theta, m, n = tt[0]
                sigmas.append(i)
                thetas.append(degrees(theta))                
        return sigmas, thetas
    
    @staticmethod
    def get_expansion_xyz(cell,cell_size):
        cell_x, cell_y, cell_z = cell_size
        exp_x = ceil(cell_x/norm(cell[:,0]))
        exp_y = ceil(cell_y/norm(cell[:,1]))
        exp_z = ceil(cell_z/norm(cell[:,2]))
        return exp_x, exp_y, exp_z
    
    def get_gb_files(self,axis,plane_n,hkl,delivery_point):
        basedir = os.getcwd()
        self.primitive_interface.compute_stgb(axis, plane_n, lim = 10, tol = 1e-5)
        half_cell = dot(self.primitive_interface.lattice_1, self.primitive_interface.bicrystal_U1)
        x,y,z = self.get_expansion_xyz(half_cell,self.calc_condition.cell_size)
        xhi = 2*x*norm(half_cell[:,0])


        hkl_intlist = list(map(int,hkl.split("_")))
        hkl_x, hkl_y, hkl_z = hkl_intlist
        axis_x, axis_y, axis_z = axis
        dirname = f"{axis_x}{axis_y}{axis_z}_{hkl_x}_{hkl_y}_{hkl_z}_gb"

        final_dirname = os.path.join(delivery_point,dirname)
        os.makedirs(final_dirname,exist_ok=True)
        os.chdir(final_dirname) 

        self.primitive_interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

        CNID = dot(self.primitive_interface.orient, self.primitive_interface.CNID)
        v1 = np.array([0,1.,0])*CNID[:,0][1] + np.array([0,0,1.])*CNID[:,0][2]
        v2 = np.array([0,1.,0])*CNID[:,1][1] + np.array([0,0,1.])*CNID[:,1][2]

        n1 = int(ceil(norm(v1)/0.2))
        n2 = int(ceil(norm(v2)/0.2))
        write_trans_file(v1,v2,n1,n2)

        print(hkl_intlist)
        self.calc_condition.generate_files(axis=axis,hkl=hkl_intlist,xhi=xhi)


        os.chdir(f"{basedir}")
        
    @staticmethod    
    def order_indice(test):
        test = np.array(test)
        test = np.sort(test)
        x, y, z = test
        if x!=y:
            test = np.array([z,y,x])
        else:
            pass
        return test
    
    @staticmethod
    def get_100_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        return mis_angle
    
    @staticmethod
    def get_110_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = GBFactory.order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle)  
        return mis_angle
    
    @staticmethod
    def get_111_angle(gbname):
        misangle_list = []
        baselist = []
        for i in list(itertools.permutations([1,-1,0])):
            baselist.append(list(i))     
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        return mis_angle
    
    def get_gbilst(self):
        hkl_list = list(self.gb_dict.keys())
        gblist = Grainboundary_list()
        for each_hkl in hkl_list:
            if self.axis==[1,0,0]:
                tilt_angle = GBFactory.get_100_angle(each_hkl)
            elif self.axis==[1,1,0]:
                tilt_angle = GBFactory.get_110_angle(each_hkl)
            else:
                tilt_angle = GBFactory.get_111_angle(each_hkl)
            
            gblist.resister(hkl=each_hkl,tilt_angle=tilt_angle)
        
        self.gblist = gblist

In [11]:
def get_100_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    return mis_angle

In [12]:
def get_110_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = GBfactory.order_indice(gbname)
    x, y, z = gbname
    tang = z/np.sqrt(x**2 + y**2)
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    mis_angle = abs(mis_angle)  
    return mis_angle

In [13]:
def get_111_angle(gbname):
    misangle_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))     
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = [-gbname[0],gbname[1],gbname[2]]
    misorientation_list = []
    for base in baselist:
        cos = dot(gbname,base)/(norm(gbname)*norm(base))
        mis_orientation = 2*(np.arccos(cos)/np.pi*180)
        mis_orientation =  abs(mis_orientation)
        misorientation_list.append(mis_orientation)
    mis_angle = min(misorientation_list)
    return mis_angle

In [14]:
class Bcc:
    @staticmethod
    def vanishing_rule(hkl):
        print(hkl)
        x, y, z = hkl
        if ((x + y + z)%2==1):
            one_two = 2
        else:
            one_two = 1
        return one_two

In [15]:
class Crystal_structure:
    def __init__(self,vanishing_rule):
        self.vanishing_rule = vanishing_rule

In [16]:
bcc = Crystal_structure(Bcc.vanishing_rule)

In [17]:
class Fcc:
    @staticmethod
    def vanishing_rule(hkl):
        x, y, z = hkl
        if ((x%2==1) & (y%2==1) & (z%2==1)):
            one_two = 1    
        else:
            one_two = 2
        return one_two

In [18]:
fcc = Crystal_structure(Fcc.vanishing_rule)

In [22]:
element = "Ag"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ag_meam = Element2(element,initializing_dir,fcc)

In [23]:
element = "Al"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Al_meam = Element2(element,initializing_dir,fcc)

In [24]:
element = "Au"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Au_meam = Element2(element,initializing_dir,fcc)

In [39]:
element = "Cu"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Cu_meam = Element2(element,initializing_dir,fcc)

In [26]:
element = "Ni"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ni_meam = Element2(element,initializing_dir,fcc)

In [27]:
element = "Pb"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pb_meam = Element2(element,initializing_dir,fcc)

In [28]:
element = "Pd"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pd_meam = Element2(element,initializing_dir,fcc)

In [40]:
element = "Pt"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pt_meam = Element2(element,initializing_dir,fcc)

In [30]:
element = "Fe"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Fe_meam = Element2(element,initializing_dir,bcc)

In [31]:
element = "Mo"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Mo_meam = Element2(element,initializing_dir,bcc)

In [32]:
element = "Nb"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Nb_meam = Element2(element,initializing_dir,bcc)

In [20]:
element = "Ta"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ta_meam = Element2(element,initializing_dir,bcc)

In [34]:
element = "V"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
V_meam = Element2(element,initializing_dir,bcc)

In [19]:
element = "W"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
W_meam = Element2(element,initializing_dir,bcc)

In [21]:
fix_bulk_region = 50
middle_region = 50
start_bulk = 160
end_bulk = 165
cell_size = [100,20,20]

In [37]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ag:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Ag Ag.meam Ag
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [38]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Al:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Al Al.meam Al
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [39]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Au:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Au Au.meam Au
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [37]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Cu:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [41]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ni:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Ni Ni.meam Ni
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [42]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pb:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Pb Pb.meam Pb
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [43]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pd:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Pd Pd.meam Pd
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Pd

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Pd crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [38]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Pt Pt.meam Pt
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [45]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Fe:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Fe Fe.meam Fe
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [46]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Mo:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Mo Mo.meam Mo
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [47]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Nb:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Nb Nb.meam Nb
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [22]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ta:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Ta Ta.meam Ta
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ta

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ta crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [49]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_V:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam V V.meam V
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [50]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_W:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam W W.meam W
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Ag

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Ag crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [ ]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Pt Pt.meam Pt
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Pt

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Pt crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [51]:
element_name_list = ["Ag","Al","Au","Cu","Ni","Pb","Pd","Pt","Fe","Mo","Nb","Ta","V","W"]
element_meam_list = [Ag_meam,Al_meam,Au_meam,Cu_meam,Ni_meam,Pb_meam,Pd_meam,Pt_meam,Fe_meam,Mo_meam,Nb_meam,Ta_meam,V_meam,W_meam]
sampling_MethoD_list = [SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ag,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Al,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Au,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Cu,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ni,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pb,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pd,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Fe,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Mo,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Nb,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ta,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_V,SamplingMethodCreator_deletelayer_MEAM_shift_energytot_W]

In [56]:
axis = [1,1,1]
max_sigma = 100

In [57]:
for element, element_meam, sampling_MethoD in zip(element_name_list,element_meam_list,sampling_MethoD_list):
    calc_condition = Calc_Condition(element=element_meam,SamplingMethod=sampling_MethoD,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
    gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)
    for hkl in list(gbfactory.gb_dict.keys()):
        gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/get_run/3/{element}")    

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[4.07293504 4.07293504 4.07293504]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504 4.07293504 4.07293504] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[4.07293504 4.07293504 4.07293504]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504 4.07293504 4.07293504] 

[[ 1.04754432e+02  1.92041545e-15  1.27991097e-16]
 [ 1.74895079e-16  2.11635913e+01 -8.75680620e-16]
 [-9.54175948e-17 -3.31616106e-16  2.01599999e+01]]
[2, 1, 1]
6
[-1 -1 -1]
38.21321070173819
[[-2.03646752e+00 -2.03646752e+00  1.24697672e-16]
 [-2.03646752e+00  0.00000000e+00 -2.03646752e+00]
 [ 0.00000000e+00 -2.03646752e+00 -2.03646752e+00]]
[4.07293504 4.07293504 4.07293504]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 2  0  1]
 [-1 -2  0]
 [ 0  1 -2]]; sigma_1 = 7

U2 = 
[[ 2  1  0]
 [ 0 -2  1]
 [-1  0 -2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504 4.07293504 4.07293504] ; theta = 38.2132106

[-4.07293504 -2.03646752  6.10940256]
[-4.07293504 -4.07293504 -4.07293504]
[  8.14587009 -10.18233761   2.03646752]
[[ 3  1  1]
 [-1  1 -5]
 [-2  1  4]]
[[ 2  1  4]
 [ 1  1 -5]
 [-3  1  1]]
num of atoms1 = 42
[[ 1.0

[[ 1.00800000e+02 -2.40643402e-15 -7.93261615e-16]
 [ 2.08081540e-15  2.11635913e+01  3.30716981e-15]
 [ 4.92793412e-15  6.87552642e-16  3.49181441e+01]]
[8, 5, 3]
98
[-1 -1 -1]
13.173551107258923
[[-2.03646752e+00 -2.03646752e+00  1.24697672e-16]
 [-2.03646752e+00  0.00000000e+00 -2.03646752e+00]
 [ 0.00000000e+00 -2.03646752e+00 -2.03646752e+00]]
[4.07293504 4.07293504 4.07293504]
rotation matrix = [[ 0.98245614 -0.12280702  0.14035088]
 [ 0.14035088  0.98245614 -0.12280702]
 [-0.12280702  0.14035088  0.98245614]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  5  3]
 [ 1 -3  2]
 [ 1 -2 -5]]; sigma_1 = 57

U2 = 
[[ 1  5  2]
 [ 1 -2  3]
 [ 1 -3 -5]]; sigma_2 = 57

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504 4.07293504 4.07293504] ; theta = 13.17355101

[-14.25527265  -2.03646752  16.29174017]
[4.07293504 4.07293504 4.07293504]
[  6.10940256 -10.18233761   4.07293504]
[[ 8 -1 -2]
 [-1 -1  5]
 [-7 -1 -3]]
[[ 7 -1 -3]
 [ 1 -1  5]
 [-8 -1 -2]]
num of atoms1 = 114


rotation matrix = [[ 0.96774194 -0.16129032  0.19354839]
 [ 0.19354839  0.96774194 -0.16129032]
 [-0.16129032  0.19354839  0.96774194]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4 -1]
 [ 1  1 -2]
 [ 1  2  4]]; sigma_1 = 31

U2 = 
[[ 1 -4 -2]
 [ 1  2 -1]
 [ 1  1  4]]; sigma_2 = 31

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.04465078 4.04465078 4.04465078] ; theta = 17.89655103

[6 5 1]
['2_1_1', '3_2_1', '4_3_1', '5_3_2', '5_4_1', '6_5_1']
[4.04465078 4.04465078 4.04465078]
rotation matrix = [[ 0.75675676 -0.32432432  0.56756757]
 [ 0.56756757  0.75675676 -0.32432432]
 [-0.32432432  0.56756757  0.75675676]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  4]
 [ 1 -3  0]
 [ 1  4 -3]]; sigma_1 = 37

U2 = 
[[ 1 -3  4]
 [ 1  0 -3]
 [ 1  4  0]]; sigma_2 = 37

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.04465078 4.04465078 4.04465078] ; theta = 50.56999199

[7 4 3]
['2_1_1', '3_2_1', '4_3_1', '5_3_2', '5_4_1', '6_5_1', '7_4_3']
[4.04465078 4.04465078 4

[[ 1.05935882e+02 -7.16603991e-16  2.45931405e-15]
 [ 3.04394042e-15  2.10166219e+01  2.97866164e-15]
 [-2.68562911e-15 -5.36237776e-16  2.62123329e+01]]
[3, 2, 1]
14
[-1 -1 -1]
27.79577249602797
[[-2.02232539e+00 -2.02232539e+00  1.23831716e-16]
 [-2.02232539e+00  0.00000000e+00 -2.02232539e+00]
 [ 0.00000000e+00 -2.02232539e+00 -2.02232539e+00]]
[4.04465078 4.04465078 4.04465078]
rotation matrix = [[ 0.92307692 -0.23076923  0.30769231]
 [ 0.30769231  0.92307692 -0.23076923]
 [-0.23076923  0.30769231  0.92307692]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  2  2]
 [ 1 -2  1]
 [ 1 -1 -2]]; sigma_1 = 13

U2 = 
[[ 1  2  1]
 [ 1 -1  2]
 [ 1 -2 -2]]; sigma_2 = 13

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.04465078 4.04465078 4.04465078] ; theta = 27.7957724

[-8.08930155  6.06697616  2.02232539]
[-4.04465078 -4.04465078 -4.04465078]
[  4.04465078  10.11162694 -14.15627772]
[[ 1  1  7]
 [ 3  1 -5]
 [-4  1 -2]]
[[-1  1  7]
 [ 4  1 -2]
 [-3  1 -5]]
num of atoms1 = 78
[[ 

...

[[ 1.03944697e+02  1.09336187e-16 -3.86656640e-15]
 [-5.52699882e-15  2.10000000e+01 -3.37874686e-15]
 [ 1.54054316e-15  4.76576778e-15  2.14330352e+01]]
[2, 1, 1]
6
[2, 1, 1]
[1, 1, 1]
[2 0 0]
38.21321070173819
[[ 1.51554446  1.51554446 -1.51554446]
 [ 1.51554446 -1.51554446  1.51554446]
 [ 1.51554446 -1.51554446 -1.51554446]]
[3.03108891 3.03108891 3.03108891]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -2]
 [ 0 -3 -2]
 [ 0 -1 -3]]; sigma_1 = 7

U2 = 
[[ 1  0 -2]
 [ 0 -2 -3]
 [ 0  1 -2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.03108891 3.03108891 3.03108891] ; theta = 38.2132106

[ 6.06217782  3.03108891 -9.09326674]
[1.51554446 1.51554446 1.51554446]
[ 12.12435565 -15.15544456   3.03108891]
[[ 3  1 -1]
 [ 5  0  3]
 [ 4  0 -6]]
[[ 2  1 -4]
 [ 5  0 -3]
 [ 1  0 -9]]
num of atoms1 = 42
[[ 1.02071666e+02 -1.8836141

[[ 1.20024997e+02 -4.31349359e-15  5.77016747e-15]
 [-2.86433595e-15  2.10000000e+01  6.42953704e-16]
 [ 1.75249042e-15 -5.64800244e-16  5.19723484e+01]]
[8, 5, 3]
98
[8, 5, 3]
[1, 1, 1]
[2 0 0]
13.173551107258923
[[ 1.51554446  1.51554446 -1.51554446]
 [ 1.51554446 -1.51554446  1.51554446]
 [ 1.51554446 -1.51554446 -1.51554446]]
[3.03108891 3.03108891 3.03108891]
rotation matrix = [[ 0.98245614 -0.12280702  0.14035088]
 [ 0.14035088  0.98245614 -0.12280702]
 [-0.12280702  0.14035088  0.98245614]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -3 -2]
 [ 0 -8  1]
 [ 0 -1 -7]]; sigma_1 = 57

U2 = 
[[ 1 -2 -3]
 [ 0 -7 -1]
 [ 0  1 -8]]; sigma_2 = 57

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.03108891 3.03108891 3.03108891] ; theta = 13.17355101

[ 24.24871129 -21.21762238  -3.03108891]
[-1.51554446 -1.51554446 -1.51554446]
[  6.06217782   9.09326674 -15.15544456]
[[ 1 -1  5]
 [ 9  0  7]
 [-6  0  8]]
[[-1 -1  5]
 [ 6  0  8]
 [-9  0  7]]
num of atoms1 = 114
[[ 1.29452694e+0

[5 3 2]
['2_1_1', '3_2_1', '4_3_1', '5_3_2']
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.95238095 -0.19047619  0.23809524]
 [ 0.23809524  0.95238095 -0.19047619]
 [-0.19047619  0.23809524  0.95238095]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -3 -1]
 [ 1  1 -2]
 [ 1  2  3]]; sigma_1 = 21

U2 = 
[[ 1 -3 -2]
 [ 1  2 -1]
 [ 1  1  3]]; sigma_2 = 21

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 21.7867892

[5 4 1]
['2_1_1', '3_2_1', '4_3_1', '5_3_2', '5_4_1']
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.96774194 -0.16129032  0.19354839]
 [ 0.19354839  0.96774194 -0.16129032]
 [-0.16129032  0.19354839  0.96774194]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4  2]
 [ 1  1 -4]
 [ 1  2  1]]; sigma_1 = 31

U2 = 
[[ 1 -4  1]
 [ 1  2 -4]
 [ 1  1  2]]; sigma_2 = 31

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 17.89655103

[6 5 1]
['2_1_1', '3_2_1',

[[ 1.00748574e+02 -2.24319130e-15 -3.36839769e-15]
 [-7.86928397e-15  2.19200000e+01 -9.17713233e-18]
 [-4.26339212e-16 -7.08097521e-16  2.23720063e+01]]
[2, 1, 1]
6
[2, 1, 1]
[1, 1, 1]
[2 0 0]
38.21321070173819
[[ 1.58193974  1.58193974 -1.58193974]
 [ 1.58193974 -1.58193974  1.58193974]
 [ 1.58193974 -1.58193974 -1.58193974]]
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1  0]
 [ 0 -3  1]
 [ 0 -1 -2]]; sigma_1 = 7

U2 = 
[[ 1  0 -1]
 [ 0 -2 -1]
 [ 0  1 -3]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 38.2132106

[ 6.32775895  3.16387947 -9.49163842]
[-1.58193974 -1.58193974 -1.58193974]
[ 12.6555179  -15.81939737   3.16387947]
[[ 3 -1  1]
 [ 5  0 -3]
 [ 4  0  6]]
[[ 2 -1  4]
 [ 5  0  3]
 [ 1  0  9]]
num of atoms1 = 42
[[ 1.06543377e+02  7.4553

[[ 1.25283235e+02  1.84791634e-15 -9.89751296e-15]
 [-2.33233824e-15  2.19200000e+01 -3.80253878e-15]
 [-3.70665488e-15  4.11288212e-15  5.42492322e+01]]
[8, 5, 3]
98
[8, 5, 3]
[1, 1, 1]
[2 0 0]
13.173551107258923
[[ 1.58193974  1.58193974 -1.58193974]
 [ 1.58193974 -1.58193974  1.58193974]
 [ 1.58193974 -1.58193974 -1.58193974]]
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.98245614 -0.12280702  0.14035088]
 [ 0.14035088  0.98245614 -0.12280702]
 [-0.12280702  0.14035088  0.98245614]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -3 -5]
 [ 0 -8 -7]
 [ 0 -1 -8]]; sigma_1 = 57

U2 = 
[[ 1 -2 -5]
 [ 0 -7 -8]
 [ 0  1 -7]]; sigma_2 = 57

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 13.17355101

[-25.3110358   22.14715632   3.16387947]
[1.58193974 1.58193974 1.58193974]
[  6.32775895   9.49163842 -15.81939737]
[[-1  1  5]
 [-9  0  7]
 [ 6  0  8]]
[[ 1  1  5]
 [-6  0  8]
 [ 9  0  7]]
num of atoms1 = 114
[[ 1.01342966e+02  

In [ ]:
element_meam,sampling_MethoD
calc_condition = Calc_Condition(element=element_meam,SamplingMethod=sampling_MethoD,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)
for hkl in list(gbfactory.gb_dict.keys()):
    gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/get_run/3/{element}")



In [33]:
axis = [1,1,1]
max_sigma = 100

In [34]:
calc_condition_Ta = Calc_Condition(element=Ta_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Ta,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)

In [35]:
gbfactory_Ta = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Ta)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.30244355 3.30244355 3.30244355]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.30244355 3.30244355 3.30244355] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.30244355 3.30244355 3.30244355]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  1]
 [ 1  0 -2]
 [ 1  1  0]]; sigma_1 = 7

U2 = 
[[ 1 -2  0]
 [ 1  1 -2]
 [ 1  0  1]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.30244355 3.30244355 3.30244355] 

In [36]:
for hkl in list(gbfactory_Ta.gb_dict.keys()):
    gbfactory_Ta.create(hkl,delivery_point="/homenfs1/yhata/calc/get_run/17")

[2 0 0]
59.99999999999999
[[ 1.65122177  1.65122177 -1.65122177]
 [ 1.65122177 -1.65122177  1.65122177]
 [ 1.65122177 -1.65122177 -1.65122177]]
[3.30244355 3.30244355 3.30244355]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -1]
 [ 0 -2 -1]
 [ 0 -1 -2]]; sigma_1 = 3

U2 = 
[[ 1  0 -1]
 [ 0 -1 -2]
 [ 0  1 -1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.30244355 3.30244355 3.30244355] ; theta = 59.9999999

[ 6.60488709 -3.30244355 -3.30244355]
[1.65122177 1.65122177 1.65122177]
[ 4.44089210e-16 -3.30244355e+00  3.30244355e+00]
[[ 1  1 -1]
 [ 3  0 -1]
 [ 0  0 -2]]
[[-1  1 -1]
 [ 0  0 -2]
 [-3  0 -1]]
num of atoms1 = 6
[[ 1.05160921e+02 -1.91195651e-15  2.77926704e-15]
 [-3.53562544e-15  2.00200000e+01  2.05439831e-15]
 [ 2.25436256e-15  8.68660813e-16  2.33518023e+01]]
[2, 1, 1]
6
[2, 1, 1]
[1, 1, 1]
[2 0 0]
38.213210701

[[ 1.22502406e+02  3.90136013e-16 -4.07522000e-15]
 [-3.25309956e-15  2.00200000e+01  9.97698967e-15]
 [-3.60876017e-15  2.24486329e-15  5.30450979e+01]]
[7, 6, 1]
86
[7, 6, 1]
[1, 1, 1]
[2 0 0]
43.573578596523625
[[ 1.65122177  1.65122177 -1.65122177]
 [ 1.65122177 -1.65122177  1.65122177]
 [ 1.65122177 -1.65122177 -1.65122177]]
[3.30244355 3.30244355 3.30244355]
rotation matrix = [[ 0.81632653 -0.30612245  0.48979592]
 [ 0.48979592  0.81632653 -0.30612245]
 [-0.30612245  0.48979592  0.81632653]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4 -1]
 [ 0 -8  3]
 [ 0 -3 -5]]; sigma_1 = 49

U2 = 
[[ 1 -1 -4]
 [ 0 -5 -3]
 [ 0  3 -8]]; sigma_2 = 49

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.30244355 3.30244355 3.30244355] ; theta = 43.5735785

[-26.41954837  16.51221773   9.90733064]
[-1.65122177 -1.65122177 -1.65122177]
[ -6.60488709 -36.32687901  42.93176611]
[[ -3  -1 -13]
 [-11   0 -15]
 [  2   0 -24]]
[[  3  -1 -13]
 [ -2   0 -24]
 [ 11   0 -15]]
num of atoms1 = 294

[[ 1.37993148e+02 -1.19681962e-15  9.48446531e-16]
 [-5.74700477e-15  2.00200000e+01  1.22351201e-15]
 [-9.90923074e-15 -6.60954564e-16  7.96703811e+01]]
[11, 8, 3]
194
[11, 8, 3]
[1, 1, 1]


In [42]:
axis = [1,1,1]
max_sigma = 100
calc_condition_Cu = Calc_Condition(element=Cu_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Cu,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
gbfactory_Cu = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Cu)
gbfactory_Cu.create("2_1_1",delivery_point="/homenfs1/yhata/calc/get_run/18")

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.62 3.62 3.62]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  1]
 [ 0 -1  1]
 [ 1  1  1]]; sigma_1 = 3

U2 = 
[[-1 -1  1]
 [ 1  0  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.62 3.62 3.62]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 38.2132106

[3 2 1]
['2_1_1', '3_2_1']
[3.62 3.62 3.62]
rotati

[[ 1.01972258e+02  5.05337381e-16  2.52668690e-16]
 [ 1.22577901e-14  2.50800957e+01 -3.75510829e-16]
 [-3.17967801e-16  3.35667719e-16  2.04778124e+01]]
[2, 1, 1]
6


In [43]:
axis = [1,1,1]
max_sigma = 100
calc_condition_Pt = Calc_Condition(element=Pt_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
gbfactory_Pt = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Pt)
gbfactory_Pt.create("2_1_1",delivery_point="/homenfs1/yhata/calc/get_run/18")

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] 

[[ 1.00753395e+02 -1.89078173e-15 -1.02618314e-15]
 [-1.15722623e-15  2.03552597e+01 -5.29206618e-15]
 [-2.30008685e-15 -4.99075478e-16  2.21599999e+01]]
[2, 1, 1]
6


In [26]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Pt Pt.meam Pt
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Pt

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Pt crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [27]:
calc_condition_Pt = Calc_Condition(element=Pt_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Pt,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)

In [28]:
gbfactory_Pt = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Pt)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] 

In [29]:
for hkl in list(gbfactory_Pt.gb_dict.keys()):
    gbfactory_Pt.create(hkl,delivery_point="/homenfs1/yhata/calc/get_run/3")

[-1 -1 -1]
59.99999999999999
[[-1.95868578e+00 -1.95868578e+00  1.19934913e-16]
 [-1.95868578e+00  0.00000000e+00 -1.95868578e+00]
 [-4.37456136e-16 -1.95868578e+00 -1.95868578e+00]]
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[ 0  1 -1]
 [ 1  0 -1]
 [-1 -1 -1]]; sigma_1 = 3

U2 = 
[[-1  0 -1]
 [ 1  1 -1]
 [ 0 -1 -1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 59.9999999

[-3.91737156  1.95868578  1.95868578]
[-3.91737156 -3.91737156 -3.91737156]
[-2.22044605e-16  1.95868578e+00 -1.95868578e+00]
[[ 1  1  1]
 [ 1  1 -1]
 [-2  1  0]]
[[-1  1  1]
 [ 2  1  0]
 [-1  1 -1]]
num of atoms1 = 6
[[ 1.00753395e+02 -1.89078173e-15 -1.02618314e-15]
 [-1.15722623e-15  2.03552597e+01 -5.29206618e-15]
 [-2.30008685e-15 -4.99075478e-16  2.21599999e+01]]
[2, 1, 1]


[[ 1.08984628e+02  1.35192919e-16 -3.99184578e-15]
 [-2.00546352e-15  2.03552597e+01  1.18621420e-16]
 [-1.25369388e-15  3.05936083e-15  3.14611522e+01]]
[7, 6, 1]
86
[-1 -1 -1]
43.573578596523625
[[-1.95868578e+00 -1.95868578e+00  1.19934913e-16]
 [-1.95868578e+00  0.00000000e+00 -1.95868578e+00]
 [-4.37456136e-16 -1.95868578e+00 -1.95868578e+00]]
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.81632653 -0.30612245  0.48979592]
 [ 0.48979592  0.81632653 -0.30612245]
 [-0.30612245  0.48979592  0.81632653]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4 -4]
 [ 1  4 -1]
 [ 1  1  4]]; sigma_1 = 49

U2 = 
[[ 1 -4 -1]
 [ 1  1 -4]
 [ 1  4  4]]; sigma_2 = 49

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 43.5735785

[15.66948623 -9.79342889 -5.87605734]
[3.91737156 3.91737156 3.91737156]
[  3.91737156  21.54554357 -25.46291513]
[[ -3  -1  13]
 [ -5  -1 -11]
 [  8  -1  -2]]
[[  3  -1  13]
 [ -8  -1  -2]
 [  5  -1 -11]]
num of

[[ 1.09125344e+02  1.50465507e-15 -6.31816331e-15]
 [-5.68296559e-16  2.03552597e+01  1.03414732e-14]
 [-3.60491072e-15 -2.93448136e-15  4.72526601e+01]]
[11, 8, 3]
194


In [95]:
class SamplingMethodCreator_cnid_MEAM_energytot_Cu:
    def __init__(self,axis,hkl):
        self.axis = axis
        self.hkl = hkl
        
    def calc_condition(self):
        calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu 
neighbor 2.0 bin 
neigh_modify delay 10 check yes 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{new_gbe}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition        

    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [96]:
axis = [1,1,1]

In [97]:
element = "Cu"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Cu_meam = Element2(element,initializing_dir,fcc)

In [98]:
calc_condition_Cu = Calc_Condition(element=Cu_meam,SamplingMethod=SamplingMethodCreator_cnid_MEAM_energytot_Cu,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)

In [99]:
gbfactory_Cu = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Cu)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.62 3.62 3.62]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  1]
 [ 0 -1  1]
 [ 1  1  1]]; sigma_1 = 3

U2 = 
[[-1 -1  1]
 [ 1  0  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.62 3.62 3.62]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 38.2132106

[3 2 1]
['2_1_1', '3_2_1']
[3.62 3.62 3.62]
rotati

In [90]:
os.getcwd()

'/homenfs1/yhata/calc/220624'

In [89]:
os.chdir("/homenfs1/yhata/calc/220624")

In [100]:
for hkl in list(gbfactory_Cu.gb_dict.keys()):
    gbfactory_Cu.create(hkl,delivery_point="/homenfs1/yhata/calc/get_run/4")

[-1 -1 -1]
59.99999999999999
[[-1.81000000e+00 -1.81000000e+00  1.10830535e-16]
 [-1.81000000e+00  0.00000000e+00 -1.81000000e+00]
 [ 0.00000000e+00 -1.81000000e+00 -1.81000000e+00]]
[3.62 3.62 3.62]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[ 0  1 -1]
 [ 1 -1 -1]
 [-1  0 -1]]; sigma_1 = 3

U2 = 
[[-1  1 -1]
 [ 1  0 -1]
 [ 0 -1 -1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 59.9999999

[ 1.81  1.81 -3.62]
[3.62 3.62 3.62]
[-1.81  1.81  0.  ]
[[-2 -1  0]
 [ 1 -1 -1]
 [ 1 -1  1]]
[[-1 -1  1]
 [-1 -1 -1]
 [ 2 -1  0]]
num of atoms1 = 6
[[ 1.01972258e+02  5.05337381e-16  2.52668690e-16]
 [ 1.22577901e-14  2.50800957e+01 -3.75510829e-16]
 [-3.17967801e-16  3.35667719e-16  2.04778124e+01]]
[2, 1, 1]
[-1 -1 -1]
38.21321070173819
[[-1.81000000e+00 -1.81000000e+00  1.10830535e-16]
 [-1.81000000e+00  0.0000

[[ 1.00711497e+02  1.80484505e-15  1.78450230e-15]
 [-2.96457672e-15  2.50800957e+01 -1.93326856e-15]
 [-5.07374676e-15  3.01416642e-15  2.90729049e+01]]
[7, 6, 1]
[-1 -1 -1]
43.573578596523625
[[-1.81000000e+00 -1.81000000e+00  1.10830535e-16]
 [-1.81000000e+00  0.00000000e+00 -1.81000000e+00]
 [ 0.00000000e+00 -1.81000000e+00 -1.81000000e+00]]
[3.62 3.62 3.62]
rotation matrix = [[ 0.81632653 -0.30612245  0.48979592]
 [ 0.48979592  0.81632653 -0.30612245]
 [-0.30612245  0.48979592  0.81632653]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4 -4]
 [ 1  4 -1]
 [ 1  1  4]]; sigma_1 = 49

U2 = 
[[ 1 -4 -1]
 [ 1  1 -4]
 [ 1  4  4]]; sigma_2 = 49

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 43.5735785

[-14.47999998   9.04999999   5.42999999]
[-3.62 -3.62 -3.62]
[  3.62        19.90999997 -23.52999997]
[[  3   1  13]
 [  5   1 -11]
 [ -8   1  -2]]
[[ -3   1  13]
 [  8   1  -2]
 [ -5   1 -11]]
num of atoms1 = 294
[[ 1.07508515e+02 -1.50561661e-15 -1.

[[ 1.00841531e+02  7.32327733e-15  3.09007711e-16]
 [ 2.46188060e-15  2.50800957e+01  4.88838521e-15]
 [-3.36244065e-15 -2.95952671e-15  4.36656638e+01]]
[11, 8, 3]


In [80]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Cu:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Cu

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [48]:
Cu_110 = ["5_5_2","9_9_4","6_1_1","8_1_1","9_2_2"]
Cu_111 = ["2_1_1"]
Mo_100 = ["8_3_0","13_5_0","5_2_0"]
Mo_110 = ["5_1_1","6_1_1","8_1_1","9_1_1"]
Mo_111 = ["4_3_1","3_2_1","11_7_4"]

In [41]:
calc_condition_Cu = Calc_Condition(element=Cu_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Cu,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)

In [45]:
max_sigma = 100
axis = [1,1,1]

In [46]:
gbfactory_Cu = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Cu)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.62 3.62 3.62]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  1]
 [ 0 -1  1]
 [ 1  1  1]]; sigma_1 = 3

U2 = 
[[-1 -1  1]
 [ 1  0  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.62 3.62 3.62]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 38.2132106

[3 2 1]
['2_1_1', '3_2_1']
[3.62 3.62 3.62]
rotati

Warning!, this programme will rewrite the POSCAR file in this dir!


In [49]:
for hkl in Cu_111:
    gbfactory_Cu.create(hkl,delivery_point="/homenfs1/yhata/calc/get_run/6")

[-1 -1 -1]
59.99999999999999
[[-1.81000000e+00 -1.81000000e+00  1.10830535e-16]
 [-1.81000000e+00  0.00000000e+00 -1.81000000e+00]
 [ 0.00000000e+00 -1.81000000e+00 -1.81000000e+00]]
[3.62 3.62 3.62]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[ 0  1 -1]
 [ 1 -1 -1]
 [-1  0 -1]]; sigma_1 = 3

U2 = 
[[-1  1 -1]
 [ 1  0 -1]
 [ 0 -1 -1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 59.9999999

[ 1.81  1.81 -3.62]
[3.62 3.62 3.62]
[-1.81  1.81  0.  ]
[[-2 -1  0]
 [ 1 -1 -1]
 [ 1 -1  1]]
[[-1 -1  1]
 [-1 -1 -1]
 [ 2 -1  0]]
num of atoms1 = 6
[[ 1.01972258e+02  5.05337381e-16  2.52668690e-16]
 [ 1.22577901e-14  2.50800957e+01 -3.75510829e-16]
 [-3.17967801e-16  3.35667719e-16  2.04778124e+01]]
[2, 1, 1]
6


In [56]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Mo:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        
#         x, y, z = hkl
#         if ((x + y + z)%2==1):
#             self.one_two = 2
#         else:
#             self.one_two = 1
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two_hkl}

variable n_layer equal {self.squared_hkl}

variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_hkl equal sqrt(v_squared_hkl)

variable norm_axis equal sqrt(v_squared_axis)

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop 2 # define b 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

variable dy equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_b-1)"

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Mo Mo.meam Mo
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Mo

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Mo crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [69]:
element = "Mo"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Mo_meam = Element2(element,initializing_dir,bcc)

In [70]:
calc_condition_Mo = Calc_Condition(element=Mo_meam,SamplingMethod=SamplingMethodCreator_deletelayer_MEAM_shift_energytot_Mo,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)

In [77]:
max_sigma = 100
axis = [1,1,1]

In [78]:
gbfactory_Mo = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition_Mo)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.14655897 3.14655897 3.14655897]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  1]
 [ 0 -1  1]
 [ 1  1  1]]; sigma_1 = 3

U2 = 
[[-1 -1  1]
 [ 1  0  1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.14655897 3.14655897 3.14655897] ; theta = 59.9999999

[2 1 1]
['2_1_1']
[3.14655897 3.14655897 3.14655897]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.14655897 3.14655897 3.14655897] 

In [79]:
for hkl in Mo_111:
    gbfactory_Mo.create(hkl,delivery_point="/homenfs1/yhata/calc/get_run/6")

[2 0 0]
27.79577249602797
[[ 1.57327948  1.57327948 -1.57327948]
 [ 1.57327948 -1.57327948  1.57327948]
 [ 1.57327948 -1.57327948 -1.57327948]]
[3.14655897 3.14655897 3.14655897]
rotation matrix = [[ 0.92307692 -0.23076923  0.30769231]
 [ 0.30769231  0.92307692 -0.23076923]
 [-0.23076923  0.30769231  0.92307692]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2 -2]
 [ 0 -4 -3]
 [ 0 -1 -4]]; sigma_1 = 13

U2 = 
[[ 1 -1 -2]
 [ 0 -3 -4]
 [ 0  1 -3]]; sigma_2 = 13

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.14655897 3.14655897 3.14655897] ; theta = 27.7957724

[  9.4396769    3.14655897 -12.58623587]
[1.57327948 1.57327948 1.57327948]
[ 15.73279483 -22.02591277   6.29311793]
[[ 4  1 -2]
 [ 7  0  3]
 [ 5  0 -9]]
[[  3   1  -5]
 [  7   0  -3]
 [  2   0 -12]]
num of atoms1 = 78
[[ 1.12310559e+02 -3.71871585e-15 -2.74435027e-16]
 [-1.00612123e-15  2.18000000e+01  2.81818561e-16]
 [ 1.31186558e-16 -1.35955451e-16  2.77896563e+01]]
[4, 3, 1]
26
[4, 3, 1]
[1, 1, 1]
[2 0 0]
38.21

In [35]:
Grade = collections.namedtuple("Grade",("score","weight"))

In [36]:
class Subject(object):
    def __init__(self):
        self._grades = []
    
    def report_grade(self,score,weight):
        self._grades.append(Grade(score,weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [37]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self,name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [38]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def student(self,name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [44]:
book = Gradebook()
albert = book.student("Albert Einstein")
math = albert.subject("Math")
math.report_grade(score=80,weight=0.10)

In [54]:
Grainboundary = collections.namedtuple("Grainboundary",("hkl","tilt_angle"))

In [186]:
class Grainboundary_list(object):
    def __init__(self):
        self._grainboundarylist = []
    
    # これがセッター
    def resister(self,hkl,tilt_angle):
        self._grainboundarylist.append(Grainboundary(hkl,tilt_angle))
        
    @property
    def grainboundarylist(self):
        return self._grainboundarylist
        
    @property
    def hkl(self):
        return np.array([i.hkl for i in self._grainboundarylist])
    
    @property
    def tilt_angle(self):
        return np.array([i.tilt_angle for i in self._grainboundarylist])